In [1]:
import tensorflow as tf 
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
print(tf.__version__)

2.3.0


In [3]:
batch_size=32
seed=42
raw_train_ds=tf.keras.preprocessing.text_dataset_from_directory("Stack Overflow dataset/train",batch_size=batch_size,
                                                                validation_split=0.2,
                                                                subset='training',
                                                                seed=seed)
raw_val_ds=tf.keras.preprocessing.text_dataset_from_directory("Stack Overflow dataset/train",batch_size=batch_size,
                                                             validation_split=0.2,
                                                             subset='validation',
                                                             seed=seed)
raw_test_ds=tf.keras.preprocessing.text_dataset_from_directory('Stack Overflow dataset/test',batch_size=batch_size)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Found 8000 files belonging to 4 classes.
Using 1600 files for validation.
Found 8000 files belonging to 4 classes.


In [41]:
import string
import re
def custom_standardization(input_data):
#     print(input_data)
    input_data=tf.strings.lower(input_data)
    input_data=tf.strings.regex_replace(input_data,'<br>',' ')
    input_data=tf.strings.regex_replace(input_data,'[%s]'%re.escape(string.punctuation),'')
    return input_data

In [42]:
max_tokens=10000
seq_max_len=250
vectorization_layer=TextVectorization(standardize=custom_standardization,max_tokens=max_tokens,
                                     output_mode='int',output_sequence_length=seq_max_len)

In [43]:
raw_train_text=raw_train_ds.map(lambda x,y:x)
vectorization_layer.adapt(raw_train_text)

In [65]:
def vectorize_text(text, label):
    print(text)
#     text = tf.expand_dims(text, -1)
#     print(text)
    return vectorization_layer(text), label
# retrieve a batch (of 32 reviews and labels) from the dataset
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review,first_label))

Review tf.Tensor(b'"why do both blank objects return the same data i have created 2 objects of the class gconveyor. i want to get some data about the objects when i select them, but both return the same information. thanks for the help!../* .* file: dragobjects.blank  .*/ .import acm.graphics.*; .import acm.program.*; .import blank.awt.event.*;..public class dragobjects extends graphicsprogram { .private static final long serialversionuid = 1l;.// initializes the program .public void init() { .    //adding a few test objects.    gconveyor conv = new gconveyor(50, 44, 120); .    add(conv); .    gconveyor conv2 = new gconveyor(30, 24, 60); .    add(conv2); ..    addmouselisteners(); .    addkeylisteners(); .    } ..// called on mouse press to record the coordinates of the click */ .public void mousepressed(mouseevent e) { .    // gpoint has x and y coordinate .    last = new gpoint(e.getpoint()); .    gobj = getelementat(last);  .    //looking at stuff so to understand.    getobjectdata 

InvalidArgumentError: slice index 1 of dimension 0 out of bounds. [Op:StridedSlice] name: text_vectorization_3/strided_slice/

In [66]:
train_ds = raw_train_ds.map(lambda text_b,label_b:(vectorization_layer(text_b),label_b))
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

Tensor("args_0:0", shape=(None,), dtype=string)
Tensor("args_0:0", shape=(None,), dtype=string)


In [59]:
print(val_ds)
for text_batch,label_batch in val_ds.take(1):
    print(text_batch.shape)
    for i in range(2):
        print(text_batch[i])

<MapDataset shapes: ((None, 250), (None,)), types: (tf.int64, tf.int32)>
(32, 250)
tf.Tensor(
[2137  187   19    6   94    9 1876   12 1283   19   21  319   19   24
    4  478   11    3   34  152   13   65 2137  187   19    6   94    9
 1876   12 1283   19   21  319    1    6    2  976 1948   36 7449 1596
   36    1    1    1    1 8564    1    1    1   27    1    1 2242    9
    2 1558    6    1    1 2242    9    2 2248    6    1    1   58  104
   74   24    4  478   11    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    

In [60]:
print(val_ds)
for text_batch,label_batch in train_ds.take(1):
    print(text_batch.shape)
    for i in range(2):
        print(text_batch[i])

<MapDataset shapes: ((None, 250), (None,)), types: (tf.int64, tf.int32)>
(32, 250)
tf.Tensor(
[  16   18 1299  159    3   34   81    4  107    5 1299   12    5    1
    1    3   17    5 2454  415    7   23  153  279  415   24   40    3
  301  202 1406    1  196   15    1    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    

In [67]:
AUTO_TUNE=tf.data.experimental.AUTOTUNE
train_ds=train_ds.cache().prefetch(buffer_size=AUTO_TUNE)
val_ds=val_ds.cache().prefetch(buffer_size=AUTO_TUNE)
train_ds=test_ds.cache().prefetch(buffer_size=AUTO_TUNE)

In [73]:
embed_dim=32

model=tf.keras.Sequential([
    layers.Embedding(max_tokens+1,embed_dim),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(4)
])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),optimizer='adam',
             metrics=tf.keras.metrics.SparseCategoricalAccuracy())


In [74]:
epochs=10
model.fit(train_ds,validation_data=val_ds,epochs=epochs)

Epoch 1/10
250/250 [==============================] - 1s 5ms/step - loss: 1.3713 - sparse_categorical_accuracy: 0.3616 - val_loss: 1.3460 - val_sparse_categorical_accuracy: 0.4563
Epoch 2/10
250/250 [==============================] - 1s 4ms/step - loss: 1.3070 - sparse_categorical_accuracy: 0.5209 - val_loss: 1.2506 - val_sparse_categorical_accuracy: 0.5562
Epoch 3/10
250/250 [==============================] - 1s 4ms/step - loss: 1.1888 - sparse_categorical_accuracy: 0.6177 - val_loss: 1.1168 - val_sparse_categorical_accuracy: 0.6463
Epoch 4/10
250/250 [==============================] - 1s 4ms/step - loss: 1.0541 - sparse_categorical_accuracy: 0.6900 - val_loss: 0.9920 - val_sparse_categorical_accuracy: 0.7056
Epoch 5/10
250/250 [==============================] - 1s 4ms/step - loss: 0.9385 - sparse_categorical_accuracy: 0.7334 - val_loss: 0.8918 - val_sparse_categorical_accuracy: 0.7444
Epoch 6/10
250/250 [==============================] - 1s 4ms/step - loss: 0.8429 - sparse_categorica

In [75]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

250/250 [==============================] - 1s 4ms/step - loss: 0.5744 - sparse_categorical_accuracy: 0.8301
Loss:  0.5744057893753052
Accuracy:  0.8301249742507935


In [91]:
print(val_ds)
for text_batch,label_batch in train_ds.take(1):
    print(text_batch.shape)
    print(model(text_batch))

<PrefetchDataset shapes: ((None, 250), (None,)), types: (tf.int64, tf.int32)>
(32, 250)
tf.Tensor(
[[ 1.01864910e+00  2.72390938e+00 -2.67486960e-01 -2.90631390e+00]
 [-2.33068883e-01  5.73790908e-01 -3.83996034e+00  3.86428952e+00]
 [ 1.91842496e+00  5.93332672e+00 -4.23278904e+00 -2.69860911e+00]
 [-3.01611602e-01 -7.17531204e-01  1.40938270e+00 -4.17184591e-01]
 [ 3.27298999e+00  1.73807287e+00 -2.99554133e+00 -1.50769317e+00]
 [ 4.77523133e-02 -2.93913305e-01  5.65608144e-01 -3.66838753e-01]
 [ 9.67699349e-01 -1.03487074e-01 -1.42177179e-01 -6.83586299e-01]
 [-4.49122190e-01 -1.26241887e+00  2.83325601e+00 -1.00161743e+00]
 [-5.51959217e-01 -5.82003534e-01  1.39206052e+00 -1.88537061e-01]
 [-3.97035837e-01 -9.40023541e-01  1.24123359e+00  3.52846533e-02]
 [ 1.09772682e+00  1.98007441e+00 -3.32152456e-01 -2.24560237e+00]
 [ 1.92769969e+00  1.36691415e+00 -2.01583934e+00 -9.43649232e-01]
 [-7.74196208e-01 -6.26803279e-01  2.90414602e-01  1.01825857e+00]
 [ 2.16854244e-01  2.09291190e

In [92]:
export_model=tf.keras.Sequential([
    vectorization_layer,
    model,
    layers.Softmax()
])

export_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                    metrics=['acc'])


In [93]:
export_model.evaluate(raw_test_ds)

250/250 [==============================] - 1s 4ms/step - loss: 0.5744 - acc: 0.8301


[0.574406087398529, 0.8301249742507935]

In [98]:
for text_batch,label_batch in raw_test_ds.take(1):
    print(export_model(text_batch))
    


tf.Tensor(
[[2.66239583e-01 2.50389189e-01 2.06916839e-01 2.76454419e-01]
 [2.02704877e-01 1.93109617e-01 4.43809986e-01 1.60375550e-01]
 [6.43859152e-03 1.49128342e-03 9.17588994e-02 9.00311172e-01]
 [3.81393254e-01 3.51416081e-01 1.51539415e-01 1.15651362e-01]
 [2.65290961e-02 2.84586251e-02 9.11056325e-02 8.53906631e-01]
 [1.76378518e-01 1.43474624e-01 5.59315205e-01 1.20831653e-01]
 [5.48055470e-02 1.51517112e-02 9.08744335e-01 2.12983508e-02]
 [1.29221395e-01 4.33349870e-02 1.06597267e-01 7.20846355e-01]
 [1.19641997e-01 1.20060198e-01 2.33394261e-02 7.36958325e-01]
 [2.00306237e-01 7.98141837e-01 2.95649777e-04 1.25627313e-03]
 [8.28096494e-02 6.00787587e-02 6.37680292e-01 2.19431236e-01]
 [3.94126356e-01 1.53263047e-01 4.13073123e-01 3.95375043e-02]
 [2.78474331e-01 1.22255348e-01 5.72461784e-01 2.68085226e-02]
 [2.30965856e-02 1.00192586e-02 2.71148272e-02 9.39769268e-01]
 [8.73269200e-01 1.01383224e-01 1.08123086e-02 1.45353442e-02]
 [2.76805848e-01 3.09144437e-01 1.13467500e-

In [102]:
only_text=raw_test_ds.map(lambda x,y:x)
for text_batch in only_text.take(1):
    print(text_batch.dtype)
    print(text_batch.shape)
    print(export_model(text_batch))

<dtype: 'string'>
(32,)
tf.Tensor(
[[1.73028529e-01 1.22906744e-01 4.73713636e-01 2.30351105e-01]
 [1.29243761e-01 8.60906720e-01 4.32570605e-03 5.52387629e-03]
 [1.23685069e-01 8.15991461e-02 6.84530258e-01 1.10185534e-01]
 [2.05235809e-01 1.43402711e-01 5.28464973e-01 1.22896567e-01]
 [2.32805401e-01 2.39914507e-01 2.97490805e-01 2.29789242e-01]
 [8.53178948e-02 9.83647704e-02 3.17718953e-01 4.98598456e-01]
 [5.87741919e-02 1.17429644e-01 7.40262717e-02 7.49769866e-01]
 [8.28096494e-02 6.00787587e-02 6.37680292e-01 2.19431236e-01]
 [3.95798206e-01 5.23534656e-01 4.48908024e-02 3.57763618e-02]
 [4.24949656e-04 8.50068347e-04 1.73593464e-04 9.98551428e-01]
 [2.70912915e-01 6.54679000e-01 6.48397133e-02 9.56842396e-03]
 [1.90475494e-01 3.33747983e-01 3.33693296e-01 1.42083183e-01]
 [4.33768630e-01 1.94838479e-01 2.58717090e-01 1.12675764e-01]
 [2.38250405e-01 2.69958287e-01 2.16378525e-01 2.75412738e-01]
 [8.73269200e-01 1.01383224e-01 1.08123086e-02 1.45353442e-02]
 [5.36999106e-01 4.2

In [100]:

# 综上可知，model可以接受的是可以直接喂给模型的tensor，例如
# export_model的入口是vectorization_layer，
# 那么export_mode接受的就是一个Tensor，这个tensor是一个batch的文字，类型是str， shape=(32,), dtype=string
# 但不能直接接受only_text，因为他是一个dataset，他持有的每一个元素都是一个batch，也就是说他持有了多个batch
export_model(only_text) #报错，而evaluate可以接受dataset



Consider rewriting this model with the Functional API.


ValueError: Attempt to convert a value (<MapDataset shapes: (None,), types: tf.string>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>) to a Tensor.